In [1]:
using Distributed
addprocs(44)

@everywhere using Pkg; @everywhere Pkg.activate("/mnt/bucket/people/briandd/Projects/pulse_input_DDM.jl")

In [2]:
@everywhere using pulse_input_DDM
using PyPlot, Statistics
using StatsBase
using HypothesisTests
import Pandas, LinearAlgebra
using DataFrames

In [3]:
using JLD2
using ImageFiltering

In [4]:
path = ENV["HOME"]*"/Projects/pulse_input_DDM.jl"
region = "FOF"; 
#region = "STR";
#region = "PPC";

In [5]:
dt = 1e-2
sessids, ratnames = sessids_from_region(region);

## Load Data

In [1]:
delay = 0.05

0.05

In [6]:
data = aggregate_spiking_data(path*"/data/hanks_data_sessions/all_times", sessids, ratnames, dt, delay = delay);

In [7]:
ts = -(ceil(maximum(vcat(data["cpoke_end"]...))/dt) + 1) * dt
tf = (ceil(maximum(vcat(data["cpoke_end"]...))/dt) + 1) * dt;
#ts,tf = -2.,2.

In [8]:
data = aggregate_and_append_extended_spiking_data!(data, path*"/data/hanks_data_sessions/all_times", 
        sessids, ratnames, dt, ts, tf, delay = delay);

## Filter data based on dprime

In [9]:
#compute the spikes/sec on each trial
FR = map((x,y)->map((y,z)->sum(x[y])/z,1:length(x),data["T"][y]),data["spike_counts_by_neuron"],data["trial"])

dprimes = abs.(map(i-> pulse_input_DDM.dprime(FR[i], data["pokedR"][data["trial"][i]]), 1:data["N0"]))

sorted_dprime = sort(dprimes,rev=true)
cell_index = sortperm(dprimes,rev=true);

In [10]:
filtSD = 3;
kern = reflect(KernelFactors.gaussian(filtSD,8*filtSD+1));

In [11]:
λ = map(x->padded_λ_array(x,dt,filtSD), data["spike_counts_by_neuron"]);
μ_λ = map(x-> imfilter(vec(eps().+nanmean(x,1)), kern), λ);

In [ ]:
#λ = map(x->padded_λ_array(x,dt,filtSD), data["spike_counts_by_neuron"]);
#μ_λ = map(x-> imfilter(vec(eps().+nanmean(x,1)), kern), λ);

λ_ex = map(x-> pulse_input_DDM.rate_mat_func_filt(x,dt,filtSD), data["spike_counts_stimulus_aligned_extended_by_neuron"]);
λ_ex = map(x-> x[:,11:end-9], λ_ex)
μ_λ_ex = map(x-> imfilter(vec(eps().+nanmean(x,1)), kern), λ_ex);

In [12]:
trunc_μ_λ = pulse_input_DDM.λ0_by_trial(data, μ_λ_ex, cpoke_aligned = false)

Dict{String,Array{Array{Array{Float64,1},1},1}} with 2 entries:
  "by_neuron" => Array{Array{Float64,1},1}[[[17.4789, 17.4268, 17.3638, 17.2914…
  "by_trial"  => Array{Array{Float64,1},1}[[[17.4789, 17.4268, 17.3638, 17.2914…

## generate single neuron data arrays

In [13]:
single_data = map(i-> filter_data_by_cell!(deepcopy(data), i:i), 1:data["N0"]);

In [14]:
single_trunc_μ_λ = map(i-> Dict("by_neuron" => [trunc_μ_λ["by_neuron"][i]], 
    "by_trial" => map(j-> [j], trunc_μ_λ["by_neuron"][i])), 1:data["N0"]);

## Fitting, test and pvalue single neurons

In [15]:
ΔLL = Array{Float64}(undef,data["N0"],10);

for j = 1:data["N0"]
    
    for i = 1:10

        ΔLL[j,i] = pulse_input_DDM.splitdata_train_test(single_data[j], single_trunc_μ_λ[j]; frac=0.8, dt=dt)
        
    end
    
    pvalues = mapslices(x-> pvalue(SignTest(x)), ΔLL[1:j,:], dims=2);
    
    JLD2.@save path*"/data/results/working/"*region*"_deltaLL_v2.jld" ΔLL pvalues
    
end

In [21]:
pvalues = mapslices(x-> pvalue(SignTest(x)), ΔLL, dims=2)

112×1 Array{Float64,2}:
 0.3437500000000001   
 0.3437500000000001   
 0.021484375000000014 
 0.021484375000000014 
 0.0019531250000000004
 0.0019531250000000004
 0.10937499999999994  
 1.0                  
 0.021484375000000014 
 0.021484375000000014 
 0.0019531250000000004
 0.7539062500000002   
 0.0019531250000000004
 ⋮                    
 0.0019531250000000004
 0.0019531250000000004
 0.021484375000000014 
 0.0019531250000000004
 0.021484375000000014 
 0.0019531250000000004
 0.0019531250000000004
 0.0019531250000000004
 0.0019531250000000004
 0.0019531250000000004
 0.021484375000000014 
 0.0019531250000000004

In [ ]:
#JLD2.@save path*"/data/results/working/"*region*"_deltaLL.jld" ΔLL

In [16]:
#JLD2.@load path*"/data/results/working/"*region*"_deltaLL_try2.jld" ΔLL

1-element Array{Symbol,1}:
 :ΔLL

In [16]:
#JLD2.@load path*"/data/results/working/"*region*"_deltaLL_1_60.jld" ΔLL

1-element Array{Symbol,1}:
 :ΔLL

In [ ]:
#blah = hcat(deepcopy(ΔLL)...)'

In [20]:
#ΔLL[1:60,:] = blah;

In [35]:
hcat(data["cellID_by_neuron"],data["sessID_by_neuron"],
    data["ratID_by_neuron"], dprimes)

BoundsError: BoundsError: attempt to access 112×4 Array{Any,2} at index [Base.LogicalIndex(Bool[false; false; true; true; true; true; false; false; true; true; true; false; true; true; true; true; false; false; true; false; true; true; true; false; false; false; false; true; true; false; false; false; false; false; true; true; true; true; false; true; false; true; true; false; true; true; false; false; false; false; false; false; true; true; false; true; true; true; true; false; false; true; true; true; true; false; true; true; true; true; false; true; false; false; true; false; true; true; true; true; true; true; true; true; false; true; false; true; false; true; false; true; true; true; true; true; true; false; false; true; true; true; true; true; true; true; true; true; true; true; true; true]), Base.Slice(Base.OneTo(1))]

In [36]:
hcat(data["cellID_by_neuron"], data["sessID_by_neuron"],
    data["ratID_by_neuron"], dprimes, pvalues)

112×5 Array{Any,2}:
 1287   46331  "B068"  0.0334404  0.34375   
 1293   46331  "B068"  0.044257   0.34375   
 1338   46484  "B068"  0.0602283  0.0214844 
 1343   46484  "B068"  0.0304013  0.0214844 
 1359   46630  "B068"  0.0390092  0.00195313
 1361   46630  "B068"  0.0303393  0.00195313
 1448   47745  "B068"  0.0134181  0.109375  
 1467   48117  "B068"  0.0346861  1.0       
 1469   48117  "B068"  0.0949105  0.0214844 
 1522   48988  "B068"  0.0197945  0.0214844 
 1533   48988  "B068"  0.0773042  0.00195313
 1535   49313  "B068"  0.0441363  0.753906  
 1553   49380  "B068"  0.0362041  0.00195313
    ⋮                                       
 7758  196708  "T063"  0.583642   0.00195313
 7759  196708  "T063"  0.335635   0.00195313
 7765  197075  "T063"  0.0986504  0.0214844 
 7766  197075  "T063"  0.447877   0.00195313
 7767  197075  "T063"  0.0628445  0.0214844 
 7772  197212  "T063"  0.0762005  0.00195313
 7774  197212  "T063"  0.100325   0.00195313
 7911  197479  "T063"  0.0774272  0